In [2]:
import os
import re
import pandas as pd
from sortedcontainers import SortedList
from collections import Counter

In [9]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
rootdir = 'C:\\Users\\Andrew\\Documents\\Python\\SpringBoard\\helpscout\\78620\\Capital-Bikeshare-Rebalancing-master'
main_dir = 'C:\\Users\\Andrew\\Documents\\Python\\SpringBoard\\helpscout\\78620\\Capital-Bikeshare-Rebalancing-master'
datadir = 'C:\\Users\\Andrew\\Documents\\Python\\SpringBoard\\helpscout\\78620\\Capital-Bikeshare-Rebalancing-master\\data'
bar_delim = '================================================'
os.chdir(main_dir)

In [4]:
#Set the directory and new file creation
def openfile(directory, file_name):
    os.chdir(directory)
    return open(file_name, 'w')

#Write the file name, feature names, and header information to a file
def create_info_doc(write_to_file, dictionary, delim):
    #Sort the dictionary by key for chronological access
    sorted_full_data_dict = SortedList()
    sorted_full_data_dict.update(list(dictionary.keys()))
    
    for key in sorted_full_data_dict:
        write_to_file.write('File name: ' + key + '\n')
        
        write_to_file.write('Column names: ')
        for feature in dictionary[key].columns:
            write_to_file.write(feature + ', ')

        write_to_file.write('\nDF:\n' + str(dictionary[key].head()))
        write_to_file.write('\n\n' + delim + '\n\n')

#Find the number of unique feature names, counts, and write to file
def write_feature_names_counts(write_to_file, dictionary, feature_list = [], reset=False):
    if reset == True:
        feature_list = []
        
    #Sort the dictionary by key for chronological access
    sorted_full_data_dict = SortedList()
    sorted_full_data_dict.update(list(dictionary.keys()))
    
    for key in list(dictionary.keys()):
        for feature in dictionary[key].columns:
            feature_list.append(feature)
            
    write_to_file.write('Unique column names and counts:\n')
    for counted_col in Counter(feature_list):
        write_to_file.write(counted_col + ': ' + 
                              str(Counter(feature_list)[counted_col]) + '\n')

    write_to_file.write('\n\nColumn names per separate dataframe:')
    for sorted_key in sorted_full_data_dict:
        write_to_file.write('\n' + sorted_key + ': ' + 
                              str(len(dictionary[sorted_key].columns)) + '\n')
        for feature in dictionary[sorted_key].columns:
                write_to_file.write(feature + ', ')

In [15]:
full_data_dict = {}
#Read in bike share data and create a dictionary to hold all dataframes
for subdir, dirs, files in os.walk(datadir):
    #print(subdir)
    #print(dirs)
    #print(files)
    for file in files:
        #print(file)
        os.chdir(subdir)
        if '.csv' in file:
            name = re.findall(r'([0-9]+[-][Q][0-9])', file)[0]
            version = re.findall(r'([-][1-2])', file)
            if len(version) > 0:
                name += version[0]
            full_data_dict[name] = pd.read_csv(file)
            print(file)

2015-Q3-cabi-trip-history-data.csv
2015-Q4-Trips-History-Data.csv
2016-Q1-Trips-History-Data.csv
2016-Q2-Trips-History-Data.csv
2016-Q3-Trips-History-Data-1.csv
2016-Q3-Trips-History-Data-2.csv


In [16]:
#Create the first file, 'full_data_dict info - ORIGINAL.txt', and write to it
full_data_dict_file = openfile(main_dir, 'full_data_dict info - ORIGINAL.txt')
create_info_doc(full_data_dict_file, full_data_dict, bar_delim)
write_feature_names_counts(full_data_dict_file, full_data_dict)
full_data_dict_file.close()

In [29]:
# look at some of the data
print pd.DataFrame(full_data_dict[name].head(2).transpose())

                                                     0                               1
Duration (ms)                                   492483                          233489
Start date                             9/30/2016 23:59                 9/30/2016 23:58
End date                                10/1/2016 0:07                  10/1/2016 0:02
Start station number                             31047                           31021
Start station                        Braddock Rd Metro  Clarendon Blvd & N Fillmore St
End station number                               31086                           31028
End station           Mount Vernon Ave & E Del Ray Ave             N Veitch & Key Blvd
Bike number                                     W20904                          W22741
Member Type                                 Registered                      Registered


In [31]:
#Create a dictionary copy for further edits
full_data_dict_edit = full_data_dict.copy()

#Partition the file names to access different dataframes independently and rename feature names
good_file_names = ['2010-Q4', '2011-Q1', '2011-Q2', '2011-Q3', '2011-Q4', '2012-Q1', '2012-Q2', '2012-Q3', '2012-Q4', '2013-Q1', '2013-Q2', '2013-Q3']
global_good_feature_names = ['Duration', 'Start Date', 'End Date', 'Start Station', 'End Station', 'Bike Number', 'Member Type']

switched_file_names = ['2013-Q4', '2014-Q1', '2014-Q2', '2014-Q3', '2014-Q4', '2015-Q1', '2015-Q2']
global_switched_feature_names = ['Duration', 'Start Date', 'Start Station', 'End Date', 'End Station', 'Bike Number', 'Member Type']

additional_features_file_names = ['2015-Q3', '2015-Q4', '2016-Q1', '2016-Q2', '2016-Q3-1', '2016-Q3-2']
global_additional_features_file_names = ['Duration', 'Start Date', 'End Date', 'Bike Number', 'Member Type', 'Start Station', 'End Station']


In [32]:
#Change the feature names based on partitions above
count = 0
for file_name in full_data_dict_edit:
    if file_name in good_file_names:
        count += 1
        full_data_dict_edit[file_name].columns = global_good_feature_names
        print(file_name, '-good-', count)                   
    elif file_name in switched_file_names:
        count += 1
        full_data_dict_edit[file_name].columns = global_switched_feature_names
        print(file_name, '-switched-', count)                   
    elif file_name in additional_features_file_names:
        count += 1
        full_data_dict_edit[file_name]['Start Station_Number'] = full_data_dict_edit[file_name]['Start station'] + ' (' + full_data_dict_edit[file_name]['Start station number'].astype(str) + ')'
        print(file_name, '-additional-', count)                   
        full_data_dict_edit[file_name].drop(['Start station', 'Start station number'], inplace=True, axis=1)
                      
        full_data_dict_edit[file_name]['End Station_Number'] = full_data_dict_edit[file_name]['End station'] + ' (' + full_data_dict_edit[file_name]['End station number'].astype(str) + ')'
        print(file_name, '-additional-', count)                   
        full_data_dict_edit[file_name].drop(['End station', 'End station number'], inplace=True, axis=1)

        full_data_dict_edit[file_name].columns = global_additional_features_file_names

('2016-Q1', '-additional-', 1)
('2016-Q1', '-additional-', 1)
('2016-Q2', '-additional-', 2)
('2016-Q2', '-additional-', 2)
('2015-Q3', '-additional-', 3)
('2015-Q3', '-additional-', 3)
('2016-Q3-2', '-additional-', 4)
('2016-Q3-2', '-additional-', 4)
('2015-Q4', '-additional-', 5)
('2015-Q4', '-additional-', 5)
('2016-Q3-1', '-additional-', 6)
('2016-Q3-1', '-additional-', 6)


In [33]:
# look at some of the data
print pd.DataFrame(full_data_dict_edit[file_name].head(2).transpose())

                                          0                       1
Duration                             840866                  656098
Start Date                  8/31/2016 23:59         8/31/2016 23:58
End Date                      9/1/2016 0:13           9/1/2016 0:09
Bike Number                          W20409                  W20756
Member Type                      Registered              Registered
Start Station  15th & Euclid St  NW (31117)  19th & G St NW (31279)
End Station           8th & H St NW (31228)   5th & K St NW (31600)


In [34]:
#Create the second file, 'full_data_dict info - EDIT.txt', and write to it
full_data_dict_file = openfile(main_dir, 'full_data_dict info - EDIT.txt')
create_info_doc(full_data_dict_file, full_data_dict_edit, bar_delim) 
write_feature_names_counts(full_data_dict_file, full_data_dict_edit, reset=True)
full_data_dict_file.close()

In [35]:
#Create a mapable regex function to find zipcodes in station features
find_zip = lambda x: re.findall(r'[0-9]+', x)

In [36]:
#Create new features in each data frame for Start Zip and End Zip
sorted_full_data_dict_edit = SortedList()
sorted_full_data_dict_edit.update(list(full_data_dict_edit.keys()))
count=0
for file in sorted_full_data_dict_edit:
    count += 1
    for feature in full_data_dict_edit[file]: 
        if feature == 'Start Station':
            full_data_dict_edit[file]['Start Zip'] = full_data_dict_edit[file][feature].astype(str).map(find_zip)
            print(file, '-start-', count)
            
        elif feature == 'End Station':
            full_data_dict_edit[file]['End Zip'] = full_data_dict_edit[file][feature].astype(str).map(find_zip)
            print(file, '-end-', count)

('2015-Q3', '-start-', 1)
('2015-Q3', '-end-', 1)
('2015-Q4', '-start-', 2)
('2015-Q4', '-end-', 2)
('2016-Q1', '-start-', 3)
('2016-Q1', '-end-', 3)
('2016-Q2', '-start-', 4)
('2016-Q2', '-end-', 4)
('2016-Q3-1', '-start-', 5)
('2016-Q3-1', '-end-', 5)
('2016-Q3-2', '-start-', 6)
('2016-Q3-2', '-end-', 6)


In [38]:
# look at some of the data
print pd.DataFrame(full_data_dict_edit['2016-Q2'].head(2).transpose())

                                                     0                                       1
Duration                                        844108                                  149995
Start Date                               4/1/2016 0:00                           4/1/2016 0:00
End Date                                 4/1/2016 0:14                           4/1/2016 0:02
Bike Number                                     W20351                                  W00443
Member Type                                     Casual                              Registered
Start Station        Jefferson Dr & 14th St SW (31247)  17th St & Massachusetts Ave NW (31267)
End Station    Columbus Circle / Union Station (31623)           17th & Corcoran St NW (31214)
Start Zip                                  [14, 31247]                             [17, 31267]
End Zip                                        [31623]                             [17, 31214]


In [39]:
#Create the third file, 'Create Zipcodes.txt', and write to it
create_zip_file = openfile(main_dir, 'Create Zipcodes.txt')
create_info_doc(create_zip_file, full_data_dict_edit, bar_delim) 
write_feature_names_counts(create_zip_file, full_data_dict_edit, reset=True)
create_zip_file.close()